# Word Embedding Visualizations.

Name: Anthony Kevin Oktavius

Here, we'll experiment with various embedding layer, with these following techniques.

* TF-IDF
* Count Vectorizer
* Word2Vec
* Neural network embedding layer.

As time passes, I'll try to add various  techniques. We also use dimensionality reduction technique to reduce the dimension of each layers, so we can visualize the relationshop between each words with matplotlib.

Library Import

In [12]:
import numpy as np
import zipfile
import gc
import pickle
import cv2
import math
import warnings
import random
import string
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
import torchvision
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import torch.utils.data as data_utils
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
# import torchmetrics
import torch.nn.functional as F
import PIL
import torch.utils.data as data_utils
import pickle
import collections

# import pytorch_lightning as pl
import imutils
import zipfile

from wordcloud import WordCloud
from PIL import Image
from torch.utils.data import Dataset
from collections import defaultdict
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python.client import device_lib
from zipfile import ZipFile
from IPython import display
from torchvision import models, transforms
from google.colab.patches import cv2_imshow
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.feature_extraction.text import CountVectorizer
from google.colab.patches import cv2_imshow

Dataset Loading with OpinRank dataset.

In [13]:
! wget https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz

--2023-09-29 04:25:32--  https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 443254787 (423M) [application/x-gzip]
Saving to: ‘enron_mail_20150507.tar.gz.1’

enron_mail_20150507 100%[===================>] 422.72M   975KB/s    in 5m 14s  

2023-09-29 04:30:46 (1.35 MB/s) - ‘enron_mail_20150507.tar.gz.1’ saved [443254787/443254787]



Unzip the dataset in tar file format.

In [14]:
! tar -xzf /content/enron_mail_20150507.tar.gz

## Data Preparation

Gather all of the email paths from all-documents folder.

In [15]:
root_path = "maildir/"
email_paths_from_all_documents_folder = glob.glob(os.path.join(root_path, "*/all_documents/*."))
print(email_paths_from_all_documents_folder[:10])

['maildir/ruscitti-k/all_documents/224.', 'maildir/ruscitti-k/all_documents/133.', 'maildir/ruscitti-k/all_documents/220.', 'maildir/ruscitti-k/all_documents/310.', 'maildir/ruscitti-k/all_documents/325.', 'maildir/ruscitti-k/all_documents/16.', 'maildir/ruscitti-k/all_documents/348.', 'maildir/ruscitti-k/all_documents/148.', 'maildir/ruscitti-k/all_documents/72.', 'maildir/ruscitti-k/all_documents/296.']


Get body email data, after X-Filename.

In [16]:
print("Sample Email data: ")

def read_file(path):
  return open(path, "r", errors='ignore')

def get_removed_components_email_text(text):
  texts_per_line = text.split("\n")

  full_email_text_per_line = []

  email_components = [
    "Message-ID: ", "Date: ", "From: ",
    "To: ", "Subject: ", "Mime-Version: ",
    "Content-Type: ", "Content-Transfer-Encoding: ", "X-From: ",
    "X-cc: ", "X-bcc: ", "X-Folder: ", "X-FileName: ",
    "X-Origin: ", "X-Filename: ", "cc: ", "@enron.com",
  ]
  NOT_FOUND = -1

  for text_per_line in text.split("\n"):
    is_exist_component = False
    for component in email_components:
      if text_per_line.find(component) != NOT_FOUND:
        is_exist_component = True
        break
    if is_exist_component == False:
      full_email_text_per_line.append(text_per_line)

  return " ".join(full_email_text_per_line)

sample_file = read_file(email_paths_from_all_documents_folder[0])
sample_body_email_data = get_removed_components_email_text(sample_file.read())

print(sample_body_email_data)

Sample Email data: 
 The attached file contains our latest daily volume requirements for 3/14 thru 3/16.           Doug Kinney Ph:  703-561-6339 Fax:  703-561-7317   - 03-14-00 Daily VolsAM.xls


Putting it all together

In [ ]:
def get_dataset():
  root_path = "maildir/"
  email_paths_from_all_documents_folder = glob.glob(os.path.join(root_path, "*/all_documents/*."))

  dataset = []

  for email_path in email_paths_from_all_documents_folder:
    file_ = read_file(email_path)
    email_body = get_removed_components_email_text(file_.read())
    dataset.append(email_body)

  return dataset

dataset = get_dataset()
print(f"Number of data: {len(dataset)}")
dataset[:10]

Number of data: 128103


[' The attached file contains our latest daily volume requirements for 3/14 thru 3/16.           Doug Kinney Ph:  703-561-6339 Fax:  703-561-7317   - 03-14-00 Daily VolsAM.xls',
 " Great to hear you'll be back soon.  Get me a call when you get back in the  states and tell that Fathead I said hi.  Kevin",
 ' Kevin,  How\'s things?  Been a while since we talked.  Hope your jobs are going well and your wedding plans are proceeding smoothly.  Did you get my e-mail about running into Mary Eyre in the gym in San Mateo? Spooky.  I\'ve seen her a couple more times but I haven\'t spoken to her; I get in the "properly pumped" zone.  Turns out there are a lot of classmates on the peninsula - David Park, Frank Knifsend, Mary, and James Regan come to mind immediately.  No real news here.  We haven\'t broken anything in the house in a while, so either we\'re getting luckier or we\'re due.  Both jobs are very busy right now so the weekends are for recuperating.  This weekend, of course, will be for w

Remove Punctuations from the dataset.

In [18]:
def get_removed_duplicated_space_in_string(my_string):
    return " ".join(my_string.split())

def get_erased_punctuations(text):
    list_of_punctuations = string.punctuation + "\‘="
    final_text = text

    for punc in list_of_punctuations:
      final_text = final_text.replace(punc, " ")

    final_text = get_removed_duplicated_space_in_string(final_text)

    return final_text

erased_punctuations_dataset = []

for datum in dataset:
  erased_punctuations_datum = get_erased_punctuations(datum)
  erased_punctuations_dataset.append(erased_punctuations_datum)

erased_punctuations_dataset[:10]

['The attached file contains our latest daily volume requirements for 3 14 thru 3 16 Doug Kinney Ph 703 561 6339 Fax 703 561 7317 03 14 00 Daily VolsAM xls',
 'Great to hear you ll be back soon Get me a call when you get back in the states and tell that Fathead I said hi Kevin',
 'Kevin How s things Been a while since we talked Hope your jobs are going well and your wedding plans are proceeding smoothly Did you get my e mail about running into Mary Eyre in the gym in San Mateo Spooky I ve seen her a couple more times but I haven t spoken to her I get in the properly pumped zone Turns out there are a lot of classmates on the peninsula David Park Frank Knifsend Mary and James Regan come to mind immediately No real news here We haven t broken anything in the house in a while so either we re getting luckier or we re due Both jobs are very busy right now so the weekends are for recuperating This weekend of course will be for watching basketball Hi to Lisa let us know how you re doing Regard

Remove integers from dataset.

In [24]:
def erase_all_digits_from_sentence(sentence):
    digits = string.digits
    table = str.maketrans('', '', digits)
    new_string = sentence.translate(table)
    return new_string

erased_all_digits_dataset = []

for datum in erased_punctuations_dataset:
  all_digits_erased_datum = erase_all_digits_from_sentence(datum)
  erased_all_digits_dataset.append(all_digits_erased_datum)

erased_all_digits_dataset[:10]

['The attached file contains our latest daily volume requirements for   thru   Doug Kinney Ph    Fax       Daily VolsAM xls',
 'Great to hear you ll be back soon Get me a call when you get back in the states and tell that Fathead I said hi Kevin',
 'Kevin How s things Been a while since we talked Hope your jobs are going well and your wedding plans are proceeding smoothly Did you get my e mail about running into Mary Eyre in the gym in San Mateo Spooky I ve seen her a couple more times but I haven t spoken to her I get in the properly pumped zone Turns out there are a lot of classmates on the peninsula David Park Frank Knifsend Mary and James Regan come to mind immediately No real news here We haven t broken anything in the house in a while so either we re getting luckier or we re due Both jobs are very busy right now so the weekends are for recuperating This weekend of course will be for watching basketball Hi to Lisa let us know how you re doing Regards Tim Sarhatt Sr Treasury Analys

Lowercase All Letters.

In [25]:
def lowercase_letters(sentence):
    return sentence.lower()

lowecase_text_dataset = []

for datum in erased_all_digits_dataset:
  lowercase_letters_datum = lowercase_letters(datum)
  lowecase_text_dataset.append(lowercase_letters_datum)

lowecase_text_dataset[:10]

['the attached file contains our latest daily volume requirements for   thru   doug kinney ph    fax       daily volsam xls',
 'great to hear you ll be back soon get me a call when you get back in the states and tell that fathead i said hi kevin',
 'kevin how s things been a while since we talked hope your jobs are going well and your wedding plans are proceeding smoothly did you get my e mail about running into mary eyre in the gym in san mateo spooky i ve seen her a couple more times but i haven t spoken to her i get in the properly pumped zone turns out there are a lot of classmates on the peninsula david park frank knifsend mary and james regan come to mind immediately no real news here we haven t broken anything in the house in a while so either we re getting luckier or we re due both jobs are very busy right now so the weekends are for recuperating this weekend of course will be for watching basketball hi to lisa let us know how you re doing regards tim sarhatt sr treasury analys

Performing Word Cloud.